
# Accediendo a Watsonx.ai vía REST API

En este laboratorio, vamos a echar un vistazo completo a la realización de peticiones HTTP para acceder a la API REST de Watsonx.ai y aprender a utilizar la funcionalidad. Este laboratorio explora sólo algunos de los muchos puntos finales REST disponibles, así que explora la documentación de la API REST para ver la lista completa de capacidades.

Antes de empezar, debe disponer de los elementos necesarios para acceder a watsonx.ai mediante programación, a saber

- tu clave de API de IBM Cloud
- la URL de servicio de IBM Cloud vinculada a su instancia
- el ID de proyecto asociado a su instancia de watsonx

In [ ]:
# primero empezaremos instalando algunas dependencias
import sys
!{sys.executable} -m pip install -q requests
!{sys.executable} -m pip install -q ibm-cloud-sdk-core
!{sys.executable} -m pip install -q ibm-watson-machine-learning==1.0.311

In [2]:
# y verificando que se han instalado correctamente
import json
import requests
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

## Cabeceras de solicitud HTTP

Las cabeceras contienen valores de parámetros que representan los metadatos asociados a las solicitudes y respuestas de una API. En el siguiente ejemplo, la cabecera Authorization proporciona al servidor las credenciales para validar tu acceso. Watsonx.ai utiliza un token de acceso "Bearer" que se utiliza para pasar nuestra clave de autenticación Watsonx.ai. La cabecera `Content-type` de la petición se añade para indicar al servidor o al navegador que está sirviendo el recurso al usuario final el tipo de medio de la petición. En este caso, el tipo de datos esperados es `application/json`.

In [ ]:
# ESTOS SON LOS VALORES QUE TENDRÁ QUE RELLENAR USTED MISMO
# clave API que ha creado
api_key = "INSERTE AQUÍ SU CLAVE"

# ID del proyecto de su instancia watsonx
project_id = "INSERTE AQUÍ EL IDENTIFICADOR DE SU PROYECTO"

# Punto final del servicio URL
ibm_cloud_url = "INSERTE AQUÍ LA URL DE SU SERVICIO"

access_token = ''
try:
  access_token = IAMTokenManager(
    apikey = api_key,
    url = "https://iam.cloud.ibm.com/identity/token"
  ).get_token()
except:
  print('Problema para obtener el token de acceso. ¿Comprobar variables?')

# Las cabeceras que enviaremos para nuestras llamadas POST y GET
headers = {
  "Authorization": "Bearer " + access_token,
  "Content-Type": "application/json",
  "Accept": "application/json"
}

## POST vs GET

Las peticiones HTTP son de dos tipos: GET y POST. Cuando se utiliza GET, los parámetros de datos se incluyen en la URL y son visibles para todos. Sin embargo, cuando se utiliza POST, los datos no se muestran en la URL, sino que se pasan en el cuerpo del mensaje HTTP.

Las peticiones GET están pensadas para recuperar datos de un servidor y no modifican su estado. Por otro lado, las peticiones POST se utilizan para enviar datos al servidor para su procesamiento y pueden modificar el estado del servidor.

## Peticiones POST con el endpoint "Generar"
El punto final de generación "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text" proporciona una interfaz para el envío de mensajes a cualquier modelo soportado por Watsonx.ai. Dado un texto como entrada, y los parámetros requeridos, el modelo seleccionado intentará completar la entrada proporcionada y devolverá "texto_generado".

El cuerpo de la solicitud debe incluir
- Model id (cadena): id del modelo
- Entrada (cadena): solicitud para generar la compleción
- Parámetros del modelo (pares clave-valor)
- ID de su proyecto watsonx

> Es importante tener en cuenta que la URL del punto final de generación depende de la ubicación de su instancia watsonx.

In [4]:
# El contenido que enviaremos como parte de nuestra petición POST
body = {
  "model_id": "google/flan-ul2",
  "input": "Write a short blog post for an advanced cloud service for large language models: This service is",
  "parameters": {
    "temperature": 0,
    "max_new_tokens": 50,
    "min_new_tokens": 25
  },
  "project_id": project_id  
}

In [ ]:
version = "2023-07-07"
generation_endpoint = f"{ibm_cloud_url}/ml/v1-beta/generation/text?version={version}"

response = requests.post(url=generation_endpoint, headers=headers, json=body)
print("Respuesta JSON sin formato:\n", json.dumps(response.json(), indent=2))

## Usando llamadas GET para recuperar datos

El método GET se utiliza para recuperar información de Watsonx.ai utilizando una URL dada.

### GET /modelos

Este enpoint obtendrá la lista de todos los modelos actualmente soportados por Watsonx.ai

In [ ]:
model_endpoint = f"{ibm_cloud_url}/ml/v1-beta/foundation_model_specs?version={version}"
response = requests.get(url = model_endpoint, headers=headers)
models = response.json()['resources']

print(f"{len(models)} modelos apoyados por Watsonx.ai")

# Descomentar a continuación para ver el JSON formato
# print(json.dumps(models, indent=2))

# Embellecer la información sólo para ver los nombres de los modelos
pretty_output = lambda m: m['label']
print(json.dumps(list(map(pretty_output, models)), indent=2))

## Usando una librería Python

Ahora que hemos visto cómo interactuar con watsonx.ai a través del uso de una API REST vamos a echar un vistazo a cómo podemos interactuar con él a través de una biblioteca de Python también.

In [ ]:
# las credenciales para autenticarse con ibm cloud
# similar a las cabeceras creadas anteriormente
creds = {
  "url": ibm_cloud_url,
  "apikey": api_key 
}

In [ ]:
# una función de ayuda que permite enviar varios avisos a la vez y parámetros para ajustarlos
def send_to_watsonxai(prompts,
                    model_name="google/flan-ul2",
                    decoding_method="greedy",
                    max_new_tokens=100,
                    min_new_tokens=30,
                    temperature=1.0,
                    repetition_penalty=2.0
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompts)), "asegúrese de que ninguno de los inputs prompts está vacío"

# Instanciar parámetros para la generación de texto
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
    }


   # Instanciar un objeto proxy modelo para enviar sus llamadas
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)


    for prompt in prompts:
        print(model.generate_text(prompt))

In [ ]:
prompt = "Write a short blog post for an advanced cloud service for large language models: This service is"

# siéntete libre de probar cambiando el modelo de uno de los valores listados antes
# y prueba a cambiar también otros valores
response = send_to_watsonxai(
  prompts=[prompt],
  model_name="google/flan-ul2",
  decoding_method="greedy",
  max_new_tokens=100,
  min_new_tokens=30,
  temperature=1.0,
  repetition_penalty=2.0
)

## Conclusión

Aunque hemos utilizado Python como nuestro lenguaje de elección en este laboratorio. Es importante señalar que a través del uso de una API REST esencialmente cualquier lenguaje puede utilizar watsonx.ai mediante programación. Lo que significa que no hay límite a la forma en que los clientes eligen integrar watsonx.ai en su pila tecnológica existente.